# Software Engineering Principles


## SOLID principles

### TL;DR

- **S**: Single responsibility principle
  - A class should only have one reason to change, only one "responsibility".
  - This translates to:
    - In general, try to have smaller classes.
    - If to change one small feature you need to change multiple classes, you might be breaking this principle.
    - If to change multiple features that are not totally related you need to change the same class, you're probably breaking this principle to.
      - E.g. If you have a class that handles both saving your data to a file, and also printing the data to stdout, and generating the view of the data in HTML.

- **O**: Open/closed principle
  - Open for extension, closed for modification: You should be able to extend a class’s behavior, without modifying it.
- **L**: Liskov's substitution law
  - Functions using base classes must be able to use derived classes without knowing it.
  - This translates to: If something is working for class "Vehicle", it should still work for classes "Motorbike", "Car" or "Truck", assuming they inherit from Vehicle.
  - Another way to simplify it: If some of your child classes is overriding a method from the parent class and just leaving it empty or raising an exception, you're probably breaking this principle.
- **I**: Interface segregation principle
  - Have smaller interfaces, which will enhance reusability.
  - This is basically the same thing as the Single responsibility principle, but for interfaces
- **D**: Dependency inversion principle
  - "Program to an interface, not to a class": Depend on abstractions, not on concretions.
  - In general, an interface is like a contract, which defines what methods will be available in your objects, with no implementation details.


### Single responsibility principle

A class should only have one reason to change, only one responsibility.

First, let's define a ```Customer``` class we'll use to represent one single customer.

```python
from dataclasses import dataclass

@dataclass
class Customer:
    id: int
    name: str
    age: int
    salary: float = 1000.
```

#### Simple example

Suppose you have a class that loads the customers, and that's also responsible for displaying them:

```python
from typing import List

class CustomersManager:
    def load(self) -> List[Customer]:
        return [
            Customer(id=1, name="Jane Doe", age=40, salary=2000.),
            Customer(id=1, name="John Doe", age=25),
        ]

    def show(self, customers: List[Customer]):
        title = f"Total customers: {len(customers)}"
        print(title)
        print("=" * len(title))
        
        for c in customers:
            print(f"{c.name}: ID={c.id} - Age={c.age}")
```




And we would use it like this:

```python
cm = CustomersManager()
customers = cm.load()
cm.show(customers=customers)
```

In this example, we can already find more than one responsibility or reason to change the class:

- What if we want to load the customers from another source?
- What if we want to change how we show the customers (e.g. show more/less fields, use a different format)

So, how do we refactor this? We should split this in smaller blocks:
- One responsible for loading
- One responsible for displaying
- (Optionally), one that coordinates the previous two.

This is how the refactored solution could look:

```python
class CustomersLoader:
    def load(self) -> List[Customer]:
        return [
            Customer(id=1, name="Jane Doe", age=40, salary=2000.),
            Customer(id=1, name="John Doe", age=25),
        ]

class CustomersPrinter:
    def show(self, customers: List[Customer]):
        title = f"Total customers: {len(customers)}"
        print(title)
        print("=" * len(title))
        
        for c in customers:
            print(f"{c.name}: ID={c.id} - Age={c.age}")
```

So, now the usage would be like this:

```python
loader = CustomersLoader()
printer = CustomersPrinter()

customers = loader.load()
printer.show(customers)
```

Now, each class has a narrower, clearer scope, which helps maintainability. If we add the optional block we mentioned, we can transform our old ```CustomersManager``` into this:

```python
@dataclass
class CustomersManager:
    loader: CustomersLoader
    printer: CustomersPrinter

    def show(self):
        customers = self.loader.load()
        self.printer.show(customers=customers)
```

This would be more of a convenience way, so this "CustomersManager" is now just coordinating the different pieces, but it will not need a change as long as the "contract" stays the same. Besides, we don't need the load method anymore, so our code could look like this:

```python
loader = CustomersLoader()
printer = CustomersPrinter()
cm = CustomersManager(loader=loader, printer=printer)

cm.show()
```


Can you think of other ways of refactoring the original ```CustomersManager``` class?

What are pros / cons of the refactored approach?

### Open/Closed principle

Open for extension, closed for modification: You should be able to extend a class’s behavior, without modifying it. This is a bit abstract, so let's see an example inspired by [this article](http://joelabrahamsson.com/a-simple-example-of-the-openclosed-principle/).

```python
from dataclasses import dataclass
from typing import List

@dataclass
class Rectangle:
    width: float
    height: float

@dataclass
class AreaCalculator:
    shapes: List[Rectangle]

    def total_area(self) -> float:
        total_area = 0.
        
        for s in self.shapes:
            total_area += s.width * s.height
        
        return total_area
```

This works pretty well for now. But.. what if now our shapes might also contain circles?

```python
from dataclasses import dataclass
from typing import List, Union
import math

@dataclass
class Rectangle:
    width: float
    height: float

@dataclass
class Circle:
    radius: float

@dataclass
class AreaCalculator:
    shapes: List[Union[Rectangle, Circle]]

    def total_area(self) -> float:
        total_area = 0.
        
        for s in self.shapes:
            if isinstance(s, Rectangle):
                total_area += s.width * s.height
            elif isinstance(s, Circle):
                total_area += s.radius**2 * math.pi
        
        return total_area
```

This is an issue: every time we add a new shape, we have to modify our ```AreaCalculator```. How do we keep that class in a way that we don't need to modify it with every new shape, while still allowing new shapes to be added?

We'll first need to add an abstract class where that all shapes can implement, and use that in our AreaCalculator:

```python
from dataclasses import dataclass
from typing import List, Union
import math
from abc import ABC, abstractmethod

class Shape(ABC):
    @abstractmethod
    def area(self) -> float:
        pass

@dataclass
class Rectangle(Shape):
    width: float
    height: float

    def area(self) -> float:
        return self.width * self.height
    
@dataclass
class Circle(Shape):
    radius: float

    def area(self) -> float:
        return s.radius**2 * math.pi

@dataclass
class AreaCalculator:
    shapes: List[Shape]

    def total_area(self) -> float:
        total_area = 0.
        
        for s in self.shapes:
            total_area += s.area()

        return total_area
```

With this, the ```AreaCalculator``` is now  closed for modification (we don't need to change it anymore if we want to add a new shape) and open for extension (it can accept new shapes without need to modify it).



### Liskov's susbtitution law

You should be able to replace a class by derived class for function calls, and it should work transparently. One classical example of this issue is the [circle-ellipse](https://en.wikipedia.org/wiki/Circle-ellipse_problem) problem (or square-rectangle problem), but we'll use a different example from [this article](https://www.tomdalling.com/blog/software-design/solid-class-design-the-liskov-substitution-principle/).

- Suppose we have an app that shows birds flying around in patterns in the sky. Following the Open/Closed principle we have just seen, we create a base class ```Bird``` that abstracts the behaviour, like this:

```python
from abc import ABC, abstractmethod
from typing import Tuple


class Bird(ABC):
    def __init__(self, position: Tuple[int, int]):
        self._position = position
        self._altitude = 0

    @property
    def position(self) -> Tuple[int, int]:
        return self._position
    
    @position.setter
    def position(self, position: Tuple[int, int]):
        self._position = position
    
    @property
    def altitude(self) -> int:
        return self._altitude
    
    @altitude.setter
    def altitude(self, altitude: int):
        self._altitude = altitude

    @abstractmethod
    def draw(self):
        pass
```

And our app would have a method that might look like this:

```python
from dataclasses import dataclass
from typing import List

@dataclass
class BirdsFlyingAroundApp:
    birds: List[Bird]
    # ...
    def arrange_bird_in_pattern(self, bird: Bird, position: Tuple[int, int], altitude: int):
        bird.position = position
        bird.altitude = altitude
```

With this, we can add different types of birds to our application easily, since we applied the Open/Closed principle.

Now, due to high demand from the users, we decide to add a new bird type, ```Penguin```. But there's one issue: penguins can't fly! So, we implement our class like this:

```python
class Penguin(Bird):
    @altitude.setter
    def altitude(self, altitude: int):
        pass
    
    # ...
```

Or like this:

```python
class Penguin(Bird):
    @altitude.setter
    def altitude(self, altitude: int):
        raise CantFlyError("Penguins can't fly!")
    
    # ...
```

In the first case, our app would still work, but penguins would just be walking around in the ground, so our patterns wouldn't look right. In the second case, our app would just crash, which is also not very convenient.

In any case, we ar breaking Liskov's substitution principle: we can't replace the class ```Bird``` with the child class ```Penguin```. In practice, this means our abstraction is not correct, because not all birds can fly.

So, how do we fix this? We need to rework our abstraction to improve it. In this case, we can split our ```Bird``` class in two:

```python
from abc import ABC, abstractmethod
from typing import Tuple

class Bird(ABC):
    def __init__(self, position: Tuple[int, int]):
        self._position = position

    @property
    def position(self) -> Tuple[int, int]:
        return self._position
    
    @position.setter
    def position(self, position: Tuple[int, int]):
        self._position = position
    
    @abstractmethod
    def draw(self):
        pass

class FlyingBird(Bird):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self._altitude = 0

    @property
    def altitude(self) -> int:
        return self._altitude
    
    @altitude.setter
    def altitude(self, altitude: int):
        self._altitude = altitude
```

And now, our app should only accept ```FlyingBird``` instances. Another option would be to check whether the bird we are arranging is a FlyingBird or not:

```python
from dataclasses import dataclass
from typing import List

@dataclass
class BirdsFlyingAroundApp:
    birds: List[FlyingBird]
    # ...
    def arrange_bird_in_pattern(self, bird: FlyingBird, position: Tuple[int, int], altitude: int):
        bird.position = position
        bird.altitude = altitude
```

In case it helps you remember:

![Liskov's Substitution Principle](../images/lsp.jpg "Liskov's Substitution Principle")

### Interface segregation principle

```Many client specific interfaces are better than one general interface```

The Interface segregation principle is very similar to the single responsibility principle, but applied to interfaces. A way to see it is that having multiple, smaller interfaces brings more flexibility. Imagine we have an ```Animal``` interface that looks like this:

```python
from abc import ABC, abstractmethod


class Animal(ABC):
    @abstractmethod
    def make_sound(self):
        pass
    
    @abstractmethod
    def eat(self, food: Food)
        pass
```

And we have a ```Listener``` class such as this:

```python
class Listener:
    def listen(self, to: Animal):
        # do something with the sounds...
```

This might work pretty well, but let's say now we want to have a second interface, ```Vehicle```, that can also ```make_sound```. How would you do this?

The issue we have is that our ```Animal``` interface is too big, we should split it into smaller pieces:

```python
class SoundMaker(ABC):
    @abstractmethod
    def make_sound(self):
        pass

class Eater(ABC):
    @abstractmethod
    def eat(self, food: Food):
        pass
```

With this change, we can have the ```Animal``` and ```Vehicle``` interfaces share the ```SoundMaker``` interface:

```python
class Animal(SoundMaker, Eater):
    pass

class Vehicle(SoundMaker):
    pass
```

And our ```Listener``` class would become:

```python
class Listener:
    def listen(self, to: SoundMaker):
        # do something with the sounds...
```

In other words, try to have interfaces (or abstract classes) that group similar behaviour, so that you will be able to easily reuse them by "componsing" them together.

You can find a simpler, more concise example [here](https://dzone.com/articles/solid-principles-by-example-interface-segregation)

### Dependency inversion principle

This principle states that ```classes should depend on an abstraction rather than on a concretion```. In other words, methods in class "A" should depend on interfaces/abstract classes rather than on concrete classes.

Again, let's illustrate this with an example. Suppose we have a class called ```Customers```, which uses another class to load the data. So we might have something like this:

```python
from dataclasses import dataclass
from typing import List


@dataclass
class Customer:
    id: int
    name: str

@dataclass
class CsvLoader:
    filename: str
    def load(self) -> List[Customer]:
        # Load customers from self.filename


@dataclass
class Customers:
    loader: CsvLoader
    _customers: List[Customer] = None

    @property
    def customers(self) -> List[Customer]:
        # Load the customers only once
        if self._customers is None:
            self._customers = self.loader.load()
        
        return self._customers
```

This might be working pretty well, but what if now we want to allow a new type of loader, like ```DatabaseLoader```?

The issue is that our ```Customers``` class depends directly on a concrete class (in this case, the ```CsvLoader```). Instead, we should create an interface/abstract class and have our methods depend on that. It could look like this:

```python
from abc import ABC, abstractmethod
from typing import List


class CustomerLoader(ABC):
    @abstractmethod
    def load(self) -> List[Customer]:
        pass
```

So now, we can have multiple implementations of ```CustomerLoader```(e.g. Csv, Database or whatever we like, as long as it implements the ```load``` method.

The type hint in our ```Customers``` class would now look like this:

```python
# ...
@dataclass
class Customers:
    loader: CustomerLoader
    # ...
```

For bigger projects, it would be a good idea to even have a more general abstraction, so we could have a generic ```Loader``` interface:

```python
from abc import ABC, abstractmethod
from typing import TypeVar, Generic, List
T = TypeVar('T')


class Loader(ABC, Generic[T]):
    @abstractmethod
    def load(self) -> List[T]:
        pass

class CustomerLoader(Loader[Customer]):
    pass

# We can now define new types of loaders very easily
class TransactionLoader(Loader[Transaction]):
    pass
```



## Other principles

- [YAGNI](https://en.wikipedia.org/wiki/You_aren%27t_gonna_need_it) - You aren't gonna need it
  - Don't over-engineer or anticipate requirements
- [KISS](https://en.wikipedia.org/wiki/KISS_principle) - keep it simple, stupid
  - Simplicity should be a design goal
- [DRY](https://en.wikipedia.org/wiki/Don%27t_repeat_yourself) - Don't repeat yourself
  - As opposite to WET (Write everything twice/we enjoy typing/wate everyone's time)
    - In practice.. [Rule of three](https://en.wikipedia.org/wiki/Rule_of_three_(computer_programming))
- [MoSCoW prioritization technique](https://en.wikipedia.org/wiki/MoSCoW_method)
  - Must / Should / Could / Won't have

## Links / Resources

- [SOLID principles made easy](https://medium.com/@dhkelmendi/solid-principles-made-easy-67b1246bcdf)
- [SOLID - Wikipedia](https://en.wikipedia.org/wiki/SOLID)
- [SOLID Cheatsheet](https://www.monterail.com/hubfs/PDF%20content/SOLID_cheatsheet.pdf)
- [Clean Code in Python](https://www.amazon.com/Clean-Code-Python-Refactor-legacy/dp/1788835832). More precisely, Chapter 4 covers the SOLID principles.
- [Refactoring: Improving the design of existing code](https://www.martinfowler.com/books/refactoring.html)
- [Online catalog for refactors / code smells](https://refactoring.com/catalog/)